In [ ]:
from google.colab import drive
import os
import pandas as pd



In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Define file paths
questionnaire_folder = "/content/drive/MyDrive/San_Jose_Local_Chapter_NHANES_Dataset/NHANES_Dataset/Questionare/Questionares"  # Update with your folder path
demographic_folder = "/content/drive/MyDrive/San_Jose_Local_Chapter_NHANES_Dataset/NHANES_Dataset/Demographics"  # Update with your demographic file path


In [ ]:

# List of variables to retain from questionnaire data
questionnaire_vars = [
    "SEQN", "ALQ151", "BPQ080", "CDQ009A", "CDQ009B", "CDQ009F", "CDQ009G",
    "CDQ008", "DBQ700", "KIQ022", "KIQ025", "KIQ480", "MCQ035", "MCQ160A", "MCQ160B",
    "MCQ160C", "MCQ160D", "MCQ160E", "MCQ160F", "MCQ160M", "MCQ220", "MCQ300B",
    "MCQ300C", "MCQ300A", "SLD012", "SMQ681", "WHD010", "WHD020", "DIQ010"
]

In [ ]:

# List of variables to retain from demographic data
demographic_vars = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "RIDRETH3", "INDFMPIR"]


In [ ]:
# Step 1: Load and merge all questionnaire data on SEQN

for file in os.listdir(questionnaire_folder):
    if file.endswith('.csv'):
        file_path = os.path.join(questionnaire_folder, file)
        data = pd.read_csv(file_path, usecols=[col for col in questionnaire_vars if col in pd.read_csv(file_path, nrows=0).columns])
        #print(f"Columns in {file}: {data.columns.tolist()}")


In [ ]:
merged_questionnaire_data = None
# Dictionary to log duplicates for each file
seqn_duplicates_log = {}

# Iterate through each file
for file in os.listdir(questionnaire_folder):
    if file.endswith('.csv'):
        file_path = os.path.join(questionnaire_folder, file)

        # Read the file
        data = pd.read_csv(file_path)

        # Standardize column names
        data.columns = data.columns.str.strip().str.upper()  # Standardize to uppercase

        # Find common variables between the file and questionnaire_vars
        common_vars = [col for col in questionnaire_vars if col in data.columns and col != "SEQN"]

        # Skip the file if it has no key variables (SEQN + relevant columns)
        if not common_vars or "SEQN" not in data.columns:
            continue

        # Optionally print common_vars only when it's not empty
        if common_vars:
            print(common_vars)

        # Retain only SEQN and relevant variables
        data = data[["SEQN"] + [col for col in common_vars if col != "SEQN"]]
        print(data)
        # Check for duplicate SEQN values
        duplicate_seqns = data[data["SEQN"].duplicated()]["SEQN"].tolist()
        if duplicate_seqns:  # Log and print only if duplicates exist
            seqn_duplicates_log[file] = duplicate_seqns
            print(f"File: {file}, Duplicate SEQNs: {seqn_duplicates_log[file]}")
        else:
            seqn_duplicates_log[file] = []

        # Merge with accumulated data
        if merged_questionnaire_data is None:
            merged_questionnaire_data = data
        else:
            merged_questionnaire_data = pd.merge(
                merged_questionnaire_data,
                data,
                on="SEQN",
                how="outer",
                suffixes=("", "_DUPLICATE")  # Add suffix to duplicate columns
            )
            # Remove duplicate columns created by the merge
            merged_questionnaire_data = merged_questionnaire_data.loc[
                :, ~merged_questionnaire_data.columns.duplicated()
            ]

# Print all duplicate SEQNs across files
print("\nSummary of Duplicate SEQNs Pre-Merge:")
for file, duplicates in seqn_duplicates_log.items():
    if duplicates:
        print(f"File: {file}, Duplicates: {duplicates}")
    else:
        print(f"File: {file}, No Duplicates Found")


['ALQ151']
          SEQN  ALQ151
0     109266.0     2.0
1     109271.0     1.0
2     109273.0     2.0
3     109274.0     2.0
4     109282.0     2.0
...        ...     ...
8960  124815.0     2.0
8961  124817.0     2.0
8962  124818.0     2.0
8963  124821.0     2.0
8964  124822.0     1.0

[8965 rows x 2 columns]
['CDQ009A', 'CDQ009B', 'CDQ009F', 'CDQ009G', 'CDQ008']
          SEQN  CDQ009A  CDQ009B  CDQ009F  CDQ009G  CDQ008
0     109271.0      NaN      NaN      NaN      NaN     2.0
1     109274.0      NaN      NaN      NaN      NaN     NaN
2     109282.0      NaN      NaN      NaN      NaN     2.0
3     109283.0      NaN      NaN      NaN      NaN     1.0
4     109284.0      NaN      NaN      NaN      NaN     NaN
...        ...      ...      ...      ...      ...     ...
6428  124815.0      NaN      NaN      NaN      NaN     NaN
6429  124817.0      NaN      NaN      NaN      NaN     1.0
6430  124818.0      NaN      NaN      NaN      NaN     2.0
6431  124821.0      NaN      NaN      NaN  

In [ ]:

# Step 2: Load demographic data
for file in os.listdir(demographic_folder):
    if file.endswith('.csv'):
        file_path = os.path.join(demographic_folder, file)
    demographic_data = pd.read_csv(file_path, usecols=demographic_vars)


In [ ]:
# Step 3: Merge the questionnaire data with demographic data on SEQN
final_merged_data = pd.merge(merged_questionnaire_data, demographic_data, on="SEQN", how="inner")


In [ ]:
# Ensure the final merged data exists
if merged_questionnaire_data is not None:
    # Count non-null values for each column
    column_counts = merged_questionnaire_data.count()

    # Convert the result to a DataFrame for better visualization
    column_counts_df = column_counts.reset_index()
    column_counts_df.columns = ['Column', 'Non-Null Count']

    # Display the counts
    print(column_counts_df)

    # Optionally save the counts to a CSV file
    column_counts_df.to_csv("column_counts.csv", index=False)
else:
    print("No data to analyze.")


               Column  Non-Null Count
0                SEQN           15560
1              ALQ151            7501
2             CDQ009A              24
3             CDQ009B              71
4             CDQ009F             149
5             CDQ009G              22
6              CDQ008            1896
7              BPQ080           10195
8              DIQ010           14986
9              DBQ700           10195
10             KIQ022            9232
11             KIQ025             383
12             KIQ480            7966
13             MCQ035            2322
14            MCQ160A            9232
15            MCQ160B            9232
16            MCQ160C            9232
17            MCQ160D            9232
18            MCQ160E            9232
19            MCQ160F            9232
20            MCQ160M            9232
21             MCQ220            9232
22            MCQ300B           13217
23            MCQ300C            9232
24            MCQ300A            9232
25          

In [ ]:

# Display the merged data
# Display the merged data
print(f"Merged data shape: {final_merged_data.shape}")
print(final_merged_data.head())


Merged data shape: (15560, 46)
       SEQN  ALQ151  CDQ009A  CDQ009B  CDQ009F  CDQ009G  CDQ008  BPQ080  \
0  109263.0     NaN      NaN      NaN      NaN      NaN     NaN     NaN   
1  109264.0     NaN      NaN      NaN      NaN      NaN     NaN     NaN   
2  109265.0     NaN      NaN      NaN      NaN      NaN     NaN     NaN   
3  109266.0     2.0      NaN      NaN      NaN      NaN     NaN     1.0   
4  109267.0     NaN      NaN      NaN      NaN      NaN     NaN     2.0   

   DIQ010  DBQ700  ...  MCQ160M_DUPLICATE  MCQ220_DUPLICATE  \
0     2.0     NaN  ...                NaN               NaN   
1     2.0     NaN  ...                NaN               NaN   
2     2.0     NaN  ...                NaN               NaN   
3     2.0     3.0  ...                2.0               2.0   
4     2.0     1.0  ...                2.0               2.0   

   MCQ300B_DUPLICATE  MCQ300C_DUPLICATE  MCQ300A_DUPLICATE  RIAGENDR  \
0                NaN                NaN                NaN       1.

In [ ]:

# Optionally save the merged data to a new CSV file
output_file = "/content/drive/MyDrive/Final_Merged_Data.csv"
final_merged_data.to_csv(output_file, index=False)
